In [ ]:
# Yeni Colab notebook aç ve çalıştır:
!pip install google-cloud-bigquery pandas numpy matplotlib seaborn scipy
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# BigQuery client
PROJECT_ID = "YOUR_PROJECT_ID"
DATASET_ID = "milano_mobile_2013"
client = bigquery.Client(project=PROJECT_ID)

# NATURE METODOLOJİSİNE UYGUN VERİ KALİTESİ KONTROLLERİ

## A. Eksik Değer Analizi

In [ ]:
def nature_style_missing_analysis():
    """Nature makalesindeki gibi eksik değer analizi"""

    print("🔍 NATURE STYLE EKSİK DEĞER ANALİZİ")
    print("="*50)

    # 1. Trafik verisi eksik değerleri
    query = """
    SELECT
        'traffic' as table_name,
        COUNT(*) as total_rows,
        SUM(CASE WHEN smsin IS NULL THEN 1 ELSE 0 END) as null_smsin,
        SUM(CASE WHEN smsout IS NULL THEN 1 ELSE 0 END) as null_smsout,
        SUM(CASE WHEN callin IS NULL THEN 1 ELSE 0 END) as null_callin,
        SUM(CASE WHEN callout IS NULL THEN 1 ELSE 0 END) as null_callout,
        SUM(CASE WHEN internet IS NULL THEN 1 ELSE 0 END) as null_internet,
        ROUND(100.0 * SUM(CASE WHEN smsin IS NULL THEN 1 ELSE 0 END) / COUNT(*), 2) as pct_null_smsin
    FROM `YOUR_PROJECT_ID.milano_mobile_2013.traffic_all_days`
    """

    df_missing = client.query(query).to_dataframe()
    print("📊 TRAFİK VERİSİ EKSİK DEĞERLER:")
    print(df_missing.to_string(index=False))

    # 2. Hareket verisi eksik değerleri
    query = """
    SELECT
        'movement' as table_name,
        COUNT(*) as total_rows,
        SUM(CASE WHEN cell2Province IS NULL THEN 1 ELSE 0 END) as null_cell2province,
        SUM(CASE WHEN Province2cell IS NULL THEN 1 ELSE 0 END) as null_province2cell,
        ROUND(100.0 * SUM(CASE WHEN cell2Province IS NULL THEN 1 ELSE 0 END) / COUNT(*), 2) as pct_null_movement
    FROM `YOUR_PROJECT_ID.milano_mobile_2013.movement_all_days`
    """

    df_movement_missing = client.query(query).to_dataframe()
    print("\n📊 HAREKET VERİSİ EKSİK DEĞERLER:")
    print(df_movement_missing.to_string(index=False))

    # 3. Nature'daki gibi eksik değer pattern'i analizi
    query = """
    WITH missing_patterns AS (
      SELECT
        CellID,
        COUNT(*) as total_records,
        SUM(CASE WHEN smsin IS NULL THEN 1 ELSE 0 END) as missing_smsin,
        SUM(CASE WHEN internet IS NULL THEN 1 ELSE 0 END) as missing_internet
      FROM `YOUR_PROJECT_ID.milano_mobile_2013.traffic_all_days`
      GROUP BY CellID
    )
    SELECT
      CASE
        WHEN missing_smsin = 0 AND missing_internet = 0 THEN 'No Missing'
        WHEN missing_smsin > 0 AND missing_internet = 0 THEN 'Missing SMS Only'
        WHEN missing_smsin = 0 AND missing_internet > 0 THEN 'Missing Internet Only'
        ELSE 'Multiple Missing'
      END as missing_pattern,
      COUNT(*) as cell_count,
      ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 2) as percentage
    FROM missing_patterns
    GROUP BY missing_pattern
    ORDER BY cell_count DESC
    """

    df_patterns = client.query(query).to_dataframe()
    print("\n🎭 EKSİK DEĞER PATTERN'LERİ (Nature Style):")
    print(df_patterns.to_string(index=False))

    return df_missing, df_movement_missing, df_patterns

# Çalıştır
df_missing, df_movement_missing, df_patterns = nature_style_missing_analysis()

## B. Aykırı Değer Tespiti

In [ ]:
def nature_style_outlier_detection():
    """Nature'daki gibi aykırı değer analizi"""

    print("\n📈 NATURE STYLE AYKIRI DEĞER ANALİZİ")
    print("="*50)

    # 1. Trafik aktivite dağılımı (Nature'daki Figure 1 benzeri)
    query = """
    WITH cell_activity AS (
      SELECT
        CellID,
        SUM(COALESCE(smsin, 0) + COALESCE(smsout, 0) +
            COALESCE(callin, 0) + COALESCE(callout, 0) +
            COALESCE(internet, 0)) as total_activity
      FROM `YOUR_PROJECT_ID.milano_mobile_2013.traffic_all_days`
      GROUP BY CellID
    ),
    stats AS (
      SELECT
        APPROX_QUANTILES(total_activity, 100)[OFFSET(25)] as q1,
        APPROX_QUANTILES(total_activity, 100)[OFFSET(50)] as median,
        APPROX_QUANTILES(total_activity, 100)[OFFSET(75)] as q3,
        APPROX_QUANTILES(total_activity, 100)[OFFSET(95)] as p95,
        APPROX_QUANTILES(total_activity, 100)[OFFSET(99)] as p99
      FROM cell_activity
    )
    SELECT
      ca.total_activity,
      COUNT(*) as cell_count,
      (SELECT q1 FROM stats) as q1,
      (SELECT median FROM stats) as median,
      (SELECT q3 FROM stats) as q3,
      (SELECT p95 FROM stats) as p95,
      (SELECT p99 FROM stats) as p99
    FROM cell_activity ca
    GROUP BY ca.total_activity
    ORDER BY ca.total_activity DESC
    LIMIT 1000
    """

    try:
        df_outliers = client.query(query).to_dataframe()

        print("🔢 TRAFİK AKTİVİTE DAĞILIMI:")
        print(f"   📊 Örneklem: {len(df_outliers)} hücre")
        print(f"   📈 Medyan aktivite: {df_outliers['median'].iloc[0]:.2f}")
        print(f"   📉 Q1 (25%): {df_outliers['q1'].iloc[0]:.2f}")
        print(f"   📈 Q3 (75%): {df_outliers['q3'].iloc[0]:.2f}")
        print(f"   🚀 P95 (95%): {df_outliers['p95'].iloc[0]:.2f}")
        print(f"   💥 P99 (99%): {df_outliers['p99'].iloc[0]:.2f}")
    except Exception as e:
        print(f"   ⚠️  Sorgu hatası: {str(e)[:100]}...")
        # Alternatif sorgu
        query_simple = """
        SELECT
          APPROX_QUANTILES(total_activity, 100)[OFFSET(25)] as q1,
          APPROX_QUANTILES(total_activity, 100)[OFFSET(50)] as median,
          APPROX_QUANTILES(total_activity, 100)[OFFSET(75)] as q3,
          APPROX_QUANTILES(total_activity, 100)[OFFSET(95)] as p95,
          APPROX_QUANTILES(total_activity, 100)[OFFSET(99)] as p99,
          COUNT(*) as cell_count
        FROM (
          SELECT
            CellID,
            SUM(COALESCE(smsin, 0) + COALESCE(smsout, 0) +
                COALESCE(callin, 0) + COALESCE(callout, 0) +
                COALESCE(internet, 0)) as total_activity
          FROM `YOUR_PROJECT_ID.milano_mobile_2013.traffic_all_days`
          GROUP BY CellID
        )
        """
        df_outliers = client.query(query_simple).to_dataframe()
        print("🔢 TRAFİK AKTİVİTE DAĞILIMI (Basit):")
        print(df_outliers.to_string(index=False))

    # 2. IQR yöntemiyle aykırı değer tespiti (Nature metodolojisi)
    query = """
    WITH cell_stats AS (
      SELECT
        CellID,
        SUM(COALESCE(internet, 0)) as internet_usage
      FROM `YOUR_PROJECT_ID.milano_mobile_2013.traffic_all_days`
      GROUP BY CellID
    ),
    summary_stats AS (
      SELECT
        AVG(internet_usage) as mean_usage,
        STDDEV(internet_usage) as std_usage,
        APPROX_QUANTILES(internet_usage, 4)[OFFSET(1)] as q1,
        APPROX_QUANTILES(internet_usage, 4)[OFFSET(3)] as q3,
        COUNT(*) as total_cells
      FROM cell_stats
    ),
    z_scores AS (
      SELECT
        cs.CellID,
        cs.internet_usage,
        (cs.internet_usage - ss.mean_usage) / ss.std_usage as z_score
      FROM cell_stats cs
      CROSS JOIN summary_stats ss
    )
    SELECT
      ss.total_cells,
      SUM(CASE WHEN ABS(zs.z_score) > 3 THEN 1 ELSE 0 END) as extreme_outliers,
      SUM(CASE WHEN zs.internet_usage < (ss.q1 - 1.5*(ss.q3-ss.q1))
                OR zs.internet_usage > (ss.q3 + 1.5*(ss.q3-ss.q1))
               THEN 1 ELSE 0 END) as iqr_outliers,
      ROUND(100.0 * SUM(CASE WHEN ABS(zs.z_score) > 3 THEN 1 ELSE 0 END) / ss.total_cells, 2) as pct_extreme
    FROM z_scores zs
    CROSS JOIN summary_stats ss
    GROUP BY ss.total_cells
    """

    try:
        df_outlier_stats = client.query(query).to_dataframe()
        print("\n🎯 AYKIRI DEĞER İSTATİSTİKLERİ:")
        print(df_outlier_stats.to_string(index=False))
    except Exception as e:
        print(f"\n⚠️  Aykırı değer sorgusu hatası: {str(e)[:100]}...")
        df_outlier_stats = pd.DataFrame()

    return df_outliers, df_outlier_stats

# Tekrar çalıştır
df_outliers, df_outlier_stats = nature_style_outlier_detection()

## C. Veri Tipleri Kontrolü ve Nature Uyumu

In [ ]:
def check_data_types_nature_compatibility():
    """Veri tiplerini kontrol et ve Nature metodolojisine uygunluğunu test et"""

    print("\n🔧 VERİ TİPLERİ VE NATURE UYUMLULUĞU")
    print("="*50)

    # 1. Trafik verisi yapısı
    query = """
    SELECT
      column_name,
      data_type,
      is_nullable
    FROM `YOUR_PROJECT_ID.milano_mobile_2013`.INFORMATION_SCHEMA.COLUMNS
    WHERE table_name = 'traffic_all_days'
    ORDER BY ordinal_position
    """

    df_schema = client.query(query).to_dataframe()
    print("📋 TRAFİK VERİSİ SCHEMA:")
    print(df_schema.to_string(index=False))

    # 2. Nature metodolojisi için gerekli sütunların varlığı
    nature_required_columns = ['datetime', 'CellID', 'smsin', 'smsout', 'callin', 'callout', 'internet']
    missing_columns = [col for col in nature_required_columns
                      if col not in df_schema['column_name'].values]

    if missing_columns:
        print(f"\n⚠️  UYARI: Nature metodolojisi için eksik sütunlar: {missing_columns}")
    else:
        print("\n✅ MÜKEMMEL: Tüm Nature metodolojisi sütunları mevcut!")

    # 3. Zaman serisi bütünlüğü kontrolü (Nature'daki temporal coverage)
    query = """
    SELECT
      DATE(datetime) as date,
      COUNT(DISTINCT CellID) as unique_cells,
      COUNT(*) as total_records,
      MIN(datetime) as first_timestamp,
      MAX(datetime) as last_timestamp,
      TIMESTAMP_DIFF(MAX(datetime), MIN(datetime), HOUR) as coverage_hours
    FROM `YOUR_PROJECT_ID.milano_mobile_2013.traffic_all_days`
    GROUP BY date
    ORDER BY date
    """

    df_temporal = client.query(query).to_dataframe()
    print("\n🕐 ZAMANSAL KAPSAM ANALİZİ (Nature Temporal Coverage):")
    print(df_temporal.to_string(index=False))

    # Nature'daki 5 aylık veri ile karşılaştırma
    total_coverage_hours = df_temporal['coverage_hours'].sum()
    nature_coverage_hours = 5 * 30 * 24  # 5 ay × 30 gün × 24 saat

    print(f"\n📊 NATURE KARŞILAŞTIRMASI:")
    print(f"   📅 Bizim veri: {total_coverage_hours} saat ({total_coverage_hours/24:.1f} gün)")
    print(f"   📚 Nature veri: {nature_coverage_hours} saat (5 ay)")
    print(f"   ⚖️  Oran: {total_coverage_hours/nature_coverage_hours:.2%}")

    return df_schema, df_temporal

# Çalıştır
df_schema, df_temporal = check_data_types_nature_compatibility()

# NATURE METODOLOJİSİNE GÖRE TEMİZLEME STRATEJİSİ

## A. EKSİK DEĞER YÖNETİMİ

In [ ]:
def apply_nature_cleaning_methodology():
    """Nature makalesindeki metodolojiye göre temizleme"""

    print("🧹 NATURE METODOLOJİSİ İLE VERİ TEMİZLEME")
    print("="*50)

    # STRATEJİ 1: Nature'daki gibi sektörel temizleme
    # Nature'da farklı veri türleri için farklı stratejiler kullanılmış

    # 1. TRAFİK VERİSİ İÇİN NATURE STRATEJİSİ
    print("1️⃣ TRAFİK VERİSİ TEMİZLEME STRATEJİSİ")
    print("   Nature'daki gibi:")
    print("   - SMS verisi: Eksikleri 0 ile doldur (aktivite yok kabul et)")
    print("   - Call verisi: Eksikleri median ile doldur")
    print("   - Internet: Eksikleri 0 ile doldur (kullanım yok)")

    # 2. HAREKET VERİSİ İÇİN NATURE STRATEJİSİ
    print("\n2️⃣ HAREKET VERİSİ TEMİZLEME STRATEJİSİ")
    print("   Nature'daki 'human mobility' bölümüne göre:")
    print("   - cell2Province: Eksikleri 0 ile doldur (hareket yok)")
    print("   - Province2cell: Eksikleri 0 ile doldur")

    # 3. TEMİZLENMİŞ TABLOLAR OLUŞTUR
    print("\n3️⃣ TEMİZLENMİŞ TABLOLAR OLUŞTURULUYOR...")

    # 3.1 Temiz trafik tablosu
    clean_traffic_query = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.traffic_clean` AS
    SELECT
      datetime,
      CellID,
      countrycode,
      -- Nature'daki gibi: SMS için 0, call için median
      COALESCE(smsin, 0) as smsin_clean,
      COALESCE(smsout, 0) as smsout_clean,
      COALESCE(callin,
        PERCENTILE_CONT(callin, 0.5) OVER(PARTITION BY CellID, EXTRACT(HOUR FROM datetime))
      ) as callin_clean,
      COALESCE(callout,
        PERCENTILE_CONT(callout, 0.5) OVER(PARTITION BY CellID, EXTRACT(HOUR FROM datetime))
      ) as callout_clean,
      COALESCE(internet, 0) as internet_clean,
      load_date
    FROM `{PROJECT_ID}.{DATASET_ID}.traffic_all_days`
    """

    client.query(clean_traffic_query).result()
    print("   ✅ traffic_clean tablosu oluşturuldu")

    # 3.2 Temiz hareket tablosu
    clean_movement_query = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.movement_clean` AS
    SELECT
      datetime,
      CellID,
      provinceName,
      COALESCE(cell2Province, 0) as cell2Province_clean,
      COALESCE(Province2cell, 0) as Province2cell_clean,
      load_date
    FROM `{PROJECT_ID}.{DATASET_ID}.movement_all_days`
    """

    client.query(clean_movement_query).result()
    print("   ✅ movement_clean tablosu oluşturuldu")

    # 4. TEMİZLENMİŞ VERİ KALİTESİ KONTROLÜ
    print("\n4️⃣ TEMİZLENMİŞ VERİ KALİTESİ KONTROLÜ")

    quality_check_query = f"""
    SELECT
      'traffic_clean' as table_name,
      COUNT(*) as total_rows,
      SUM(CASE WHEN smsin_clean IS NULL THEN 1 ELSE 0 END) as null_smsin,
      SUM(CASE WHEN smsout_clean IS NULL THEN 1 ELSE 0 END) as null_smsout,
      ROUND(100.0 * SUM(CASE WHEN smsin_clean IS NULL THEN 1 ELSE 0 END) / COUNT(*), 2) as pct_null_after
    FROM `{PROJECT_ID}.{DATASET_ID}.traffic_clean`
    """

    df_quality = client.query(quality_check_query).to_dataframe()
    print("📊 TEMİZLENMİŞ VERİ KALİTESİ:")
    print(df_quality.to_string(index=False))

    return df_quality

# Temizleme uygula
df_quality = apply_nature_cleaning_methodology()

## B. AYKIRI DEĞER YÖNETİMİ

In [ ]:
def apply_nature_outlier_strategy():
    """Nature'daki aykırı değer yönetimi stratejisi"""

    print("\n🎯 NATURE AYKIRI DEĞER YÖNETİMİ")
    print("="*50)

    # Nature'daki strateji: Çok yüksek değerleri winsorize et
    print("Nature makalesindeki strateji:")
    print("1. Z-score > 3 olan değerleri winsorize et")
    print("2. IQR yöntemiyle extreme outlier'ları kırp")
    print("3. Mantıksız değerleri (negatif trafik) temizle")

    # Winsorized trafik tablosu oluştur
    winsorized_query = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.traffic_winsorized` AS
    WITH cell_stats AS (
      SELECT
        CellID,
        PERCENTILE_CONT(smsin_clean, 0.01) OVER(PARTITION BY CellID) as p1_smsin,
        PERCENTILE_CONT(smsin_clean, 0.99) OVER(PARTITION BY CellID) as p99_smsin,
        PERCENTILE_CONT(internet_clean, 0.01) OVER(PARTITION BY CellID) as p1_internet,
        PERCENTILE_CONT(internet_clean, 0.99) OVER(PARTITION BY CellID) as p99_internet
      FROM `{PROJECT_ID}.{DATASET_ID}.traffic_clean`
    )
    SELECT
      t.datetime,
      t.CellID,
      t.countrycode,
      -- Winsorize: %1 ve %99 arasına sıkıştır
      CASE
        WHEN t.smsin_clean < s.p1_smsin THEN s.p1_smsin
        WHEN t.smsin_clean > s.p99_smsin THEN s.p99_smsin
        ELSE t.smsin_clean
      END as smsin_winsorized,
      t.smsout_clean,
      t.callin_clean,
      t.callout_clean,
      CASE
        WHEN t.internet_clean < s.p1_internet THEN s.p1_internet
        WHEN t.internet_clean > s.p99_internet THEN s.p99_internet
        ELSE t.internet_clean
      END as internet_winsorized,
      t.load_date
    FROM `{PROJECT_ID}.{DATASET_ID}.traffic_clean` t
    JOIN cell_stats s ON t.CellID = s.CellID
    """

    client.query(winsorized_query).result()
    print("\n✅ traffic_winsorized tablosu oluşturuldu")

    # Winsorize etkisini kontrol et
    effect_query = f"""
    WITH original_stats AS (
      SELECT
        AVG(smsin_clean) as orig_avg_smsin,
        STDDEV(smsin_clean) as orig_std_smsin,
        MAX(smsin_clean) as orig_max_smsin
      FROM `{PROJECT_ID}.{DATASET_ID}.traffic_clean`
    ),
    winsorized_stats AS (
      SELECT
        AVG(smsin_winsorized) as win_avg_smsin,
        STDDEV(smsin_winsorized) as win_std_smsin,
        MAX(smsin_winsorized) as win_max_smsin
      FROM `{PROJECT_ID}.{DATASET_ID}.traffic_winsorized`
    )
    SELECT
      o.orig_avg_smsin,
      w.win_avg_smsin,
      o.orig_std_smsin,
      w.win_std_smsin,
      o.orig_max_smsin,
      w.win_max_smsin,
      ROUND((o.orig_std_smsin - w.win_std_smsin) / o.orig_std_smsin * 100, 2) as std_reduction_pct
    FROM original_stats o, winsorized_stats w
    """

    df_winsorize_effect = client.query(effect_query).to_dataframe()
    print("📊 WINSORIZE ETKİSİ:")
    print(df_winsorize_effect.to_string(index=False))

    return df_winsorize_effect

# Aykırı değer stratejisini uygula
df_winsorize_effect = apply_nature_outlier_strategy()

## C. NATURE ANALİZLERİ İÇİN HAZIR FİNAL TABLOLAR

In [ ]:
def create_final_analysis_tables():
    """Nature analizleri için final tablolar oluştur"""

    print("\n📊 NATURE ANALİZLERİ İÇİN FİNAL TABLOLAR")
    print("="*50)

    # 1. HÜCRE BAZLI ÖZET (Nature'daki anten özeti gibi)
    print("1️⃣ Hücre bazlı özet tablosu...")

    cell_summary_query = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.cell_summary_nature` AS
    SELECT
      CellID,
      COUNT(*) as observation_count,
      SUM(smsin_winsorized) as total_smsin,
      SUM(smsout_clean) as total_smsout,
      SUM(callin_clean) as total_callin,
      SUM(callout_clean) as total_callout,
      SUM(internet_winsorized) as total_internet,
      AVG(smsin_winsorized) as avg_smsin,
      AVG(internet_winsorized) as avg_internet,
      -- Nature'daki gibi aktivite çeşitliliği metriği
      (SUM(smsin_winsorized) + SUM(smsout_clean) +
       SUM(callin_clean) + SUM(callout_clean) +
       SUM(internet_winsorized)) as total_activity,
      -- Nature'daki gibi yoğunluk metriği
      CASE
        WHEN SUM(callin_clean) + SUM(callout_clean) > 0
        THEN (SUM(smsin_winsorized) + SUM(internet_winsorized)) /
             (SUM(callin_clean) + SUM(callout_clean))
        ELSE NULL
      END as data_voice_ratio
    FROM `{PROJECT_ID}.{DATASET_ID}.traffic_winsorized`
    GROUP BY CellID
    """

    client.query(cell_summary_query).result()
    print("   ✅ cell_summary_nature tablosu oluşturuldu")

    # 2. EYALET HAREKET ÖZETİ (Nature'daki migration analizi)
    print("\n2️⃣ Eyalet hareket özeti...")

    province_summary_query = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.province_movement_nature` AS
    SELECT
      provinceName,
      COUNT(DISTINCT CellID) as connected_cells,
      SUM(cell2Province_clean) as total_from_milano,
      SUM(Province2cell_clean) as total_to_milano,
      AVG(cell2Province_clean) as avg_from_milano,
      STDDEV(cell2Province_clean) as std_from_milano,
      -- Nature'daki gibi net göç metriği
      (SUM(cell2Province_clean) - SUM(Province2cell_clean)) as net_movement,
      -- Nature'daki gibi hareket yoğunluğu
      CASE
        WHEN COUNT(DISTINCT CellID) > 0
        THEN SUM(cell2Province_clean) / COUNT(DISTINCT CellID)
        ELSE 0
      END as movement_per_cell
    FROM `{PROJECT_ID}.{DATASET_ID}.movement_clean`
    WHERE cell2Province_clean > 0 OR Province2cell_clean > 0
    GROUP BY provinceName
    ORDER BY total_from_milano DESC
    """

    client.query(province_summary_query).result()
    print("   ✅ province_movement_nature tablosu oluşturuldu")

    # 3. ZAMAN SERİSİ ÖZETİ (Nature'daki temporal pattern)
    print("\n3️⃣ Zaman serisi özeti...")

    temporal_summary_query = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.temporal_pattern_nature` AS
    WITH hourly_data AS (
      SELECT
        EXTRACT(HOUR FROM datetime) as hour_of_day,
        EXTRACT(DAYOFWEEK FROM datetime) as day_of_week,
        CellID,
        SUM(smsin_winsorized) as hourly_smsin,
        SUM(internet_winsorized) as hourly_internet,
        SUM(callin_clean + callout_clean) as hourly_calls
      FROM `{PROJECT_ID}.{DATASET_ID}.traffic_winsorized`
      GROUP BY hour_of_day, day_of_week, CellID
    ),
    p75_calc AS (
      SELECT
        hour_of_day,
        APPROX_QUANTILES(hourly_smsin, 100)[OFFSET(75)] as p75_smsin_by_hour
      FROM hourly_data
      GROUP BY hour_of_day
    )
    SELECT
      h.hour_of_day,
      h.day_of_week,
      COUNT(DISTINCT h.CellID) as active_cells,
      SUM(h.hourly_smsin) as total_hourly_smsin,
      SUM(h.hourly_internet) as total_hourly_internet,
      SUM(h.hourly_calls) as total_hourly_calls,
      AVG(h.hourly_smsin) as avg_smsin_per_cell,
      AVG(h.hourly_internet) as avg_internet_per_cell,
      p.p75_smsin_by_hour
    FROM hourly_data h
    LEFT JOIN p75_calc p ON h.hour_of_day = p.hour_of_day
    GROUP BY h.hour_of_day, h.day_of_week, p.p75_smsin_by_hour
    ORDER BY h.day_of_week, h.hour_of_day
    """

    client.query(temporal_summary_query).result()
    print("   ✅ temporal_pattern_nature tablosu oluşturuldu")

    print("\n🎉 NATURE ANALİZLERİ İÇİN TÜM FİNAL TABLOLAR HAZIR!")

    # Tablo özeti
    tables = ['cell_summary_nature', 'province_movement_nature', 'temporal_pattern_nature']
    for table in tables:
        try:
            query = f"SELECT COUNT(*) as cnt FROM `{PROJECT_ID}.{DATASET_ID}.{table}`"
            result = client.query(query).to_dataframe()
            print(f"   📊 {table}: {result['cnt'].iloc[0]:,} satır")
        except Exception as e:
            print(f"   ⚠️  {table}: Kontrol edilemedi - {str(e)[:50]}")

# Final tabloları oluştur
create_final_analysis_tables()